# Circular Molecules

> ### In this tutorial we will cover:
> - how we can build circular structures in BuildAMol

Sometimes, the molecules we want to build are circular. This poses a challenge to BuildAMol which is tailored to linear molecules. In fact, BuildAMol's rotational optimization algorithms for improving conformations are useless for circular structures. Hence, in pure BuildAMol we cannot build circular molecules at this point!

For small rings it is usually possible to find reference components that already contain these rings, allowing users to circumvent the problem altogether by smart choices of starting compounds. 

However, didn't the tutorial title suggest that we _could_ build circular structures? Well, yes, we can - it's just not something that BuildAMol _inherently_ can do. But this is where BuildAMol's excellent integratability with other libraries comes in handy. 

The cheminformatics library `RDKit` uses a molecular force-field optimization that is perfectly able to handle ring-like structures. So, in order to build circular structures in BuildAMol, we (1) create the structure we want linearly, (2) "circularize" it by adding the necessary bonds, and (3) export it to RDKit to perform the conformational optimization. If after cricularization our building process is not yet complete, we can always (4) import the molecule back into BuildAMol to continue our work.

### Going from linear to circular

Let's make a super simple circular poly-Histidine.

In [1]:
import buildamol as bam
bam.visual.set_backend("py3dmol")

In [2]:
# load the amino acids
bam.load_amino_acids()

# make a histidine
his = bam.molecule("HIS")

# and make a polymer of 20 histidines
peptide_link = bam.get_linkage("LINK") # get the peptide linkage is just called LINK in the database
his.repeat(20, peptide_link)

his.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Now that we have our linear poly-histidine let's add a "circular" bond by connecting the first residue to the last residue (don't worry about the side chains at this point). We cannot use `bam.connect` or the `attach` method for this, however, because these methods would transpose the atom coordinates in our molecule, completely twisting the structure. But it's no trouble to manually add one bond and remove two atoms without adjusting the atom coordinates. So let's do it.

In [3]:
# luckily the linkage can also be applied directly to make bonds
# without adjusting the atom placements. We simply have to specify the target and source residues (first and last)
# of the "his" molecule, which is both the source and target molecule in this case
peptide_link.apply(target=his, source=his, target_residue=-1, source_residue=1)

If we look at the residue graph again, we will now see a new very looong bond spanning the entire length of the polymer.

In [4]:
his.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Optimize with RDKit

Alright, we now have a "pseudo-circular" molecule. However, it's actual structure is still a bad one because of the impossibly long bond between the first and last residues. And all the side chains still look somewhat crooked... To fix this we can use RDKit's energy minimization. Luckily, we can use the Molecule's `optimize` method with the argument `algorithm='rdkit'` to do so:

In [5]:
# optimize the histidine-ring structure using RDKit to get a true ring structure
his = his.optimize(algorithm="rdkit")

# and show the structure
his.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
# let's save the structure to a pdb file
his.to_pdb("./files/his20.pdb")

And there it is, a nice circle of histidines!

So, that's it for this tutorial. Building circular structures is not so difficult after all, but it does involve making use of BuildAMol's integratability features. Thank you for checking out this tutorial and good luck with your research!